## Data Exploration - Client Matching Table

In [ ]:
# Load libraries
import sys
import os
import datetime 
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sqlalchemy import create_engine

In [ ]:
# Import helper functions
sys.path.append('../src/')
from helpers.helpers import get_database_connection as get_db_conn

In [ ]:
# Visualization functions

def visualize_dob_across_time(df):
    '''Takes data frame, creates barplot of births across time'''
    dob_df = df.copy()
    dob_df.dob = pd.DatetimeIndex(dob_df.dob).year
    dob_df.groupby(dob_df.dob).size().plot.bar(figsize=(20,5))

def visualize_sources(df):
    '''Takes data frame, creates barplot of rows per source'''
    plot_df = df.copy()
    plot_df.groupby(plot_df.source).size().sort_values(ascending=False).plot.bar(figsize=(10,5))

def visualize_joids(joid_df):
    '''Takes data frame, creates barplot of how often each joid appears in the table'''
    plot_df = joid_df.copy()
    plot_df["count"].plot.bar(figsize=(10,5))

def visualize_matching(df):
    '''Takes data frame, creates barplot of how well records were matched'''
    plot_df = df.copy()
    plot_df.groupby(plot_df.joidassignedby).size().sort_values(ascending=False).plot.bar(figsize=(10,5))


In [ ]:
# Set connection
db_conn = get_db_conn()
db_conn

In [ ]:
# Load data
df_q = """SELECT * FROM clean.jocojococlient;"""
df = pd.read_sql(df_q, db_conn)
df

### Overview

In [ ]:
# Null values per column
df.isna().sum()

Some records are matched through source id, others through hash_sourceid, so we would expect some nulls in these columns. 

In [ ]:
# Unique values per column
df.nunique()

* There are a total of 1,262,067 distinct joids, each of them appears 1.6 times in this table. 
* Urno and id are equivalent to the number of rows -> we should remove urno from the table. 
* We have a total of 10 sources of data (some joids match to a hash_sourceid instead of a sourceid)

In [ ]:
# Histogram - date of birth
visualize_dob_across_time(df)

There are about 5000 dates of birth default values (1900-01-01) - they need to be set to null. 

In [ ]:
df[['dob','urno']].groupby(['dob'])['urno'] \
                             .count() \
                             .reset_index(name='count') \
                             .sort_values(['count'], ascending=False) \
                             .head(10)

In [ ]:
# Rows per source
df['source'].value_counts()

In [ ]:
# Rows per source - viz
visualize_sources(df)

The source with the highest number of rows is JIMS, followed by MedACT.

In [ ]:
# joid counts (faster than through df)
joid_df_q = """select joid, count(*)
    from clean.jocojococlient
    group by joid 
    order by count(*) desc;"""
joid_df = pd.read_sql(joid_df_q, db_conn)
joid_df


Some joids are repeated more than 100 times in the table. Some of these would appear to correspond to individuals with multiple ids in the relevant source, e.g. if the source id is not at the person's level or they are not able to match well specific individuals. 

### Analysis of matching across columns

#### Matching: 
- As explained by Steve, record matching is done through 4 fields:
    * first name (yes or no)
    * last name (yes or no)
    * social security number (0, 4, or 9 numbers matched)
    * date of birth (yes or no)
The column joidassignedby summarizes how those columns were matched, and the following 4 columns (e.g. lastnamepop) show how each individual column was matched.

In [ ]:
# Show how well matching was done
visualize_matching(df)

Most records were matched at 400(1-1-049-1), 100(1-1-9-1) or 300(1-1-049-1). These seem to indicate quite accurate matches, but it is unclear what the initial number means -> ask Steve for further clarification.

#### How to match individuals across tables? 

* The jocojococlient has one joid per individual. 
* Each individual (joid) may be linked to other sources through either the sourceid or the hash_sourceid. 
* The source field indicates what column to match the sourceid/hash_sourceid to.
    * e.g. JOCOJCMHCDEMOGRAPHICS.PATID: for any MHC data, we can link a patid (id of MHC tables) to a joid. This means that we need to match both source = 'JOCOJCMHCDEMOGRAPHICS.PATID' AND sourceid = 'patid'
    * e.g. JOCOMEDACTINCIDENTS.RCDID: in the case of MedACT data, we actually matched to the hash_id, meaning we need to match both source = 'JOCOMEDACTINCIDENTS.RCDID' AND hash_sourceid = hash_rcdid
* Each joid can be linked to multiple sources. 
* Each joid can be linked to multiple sourceid/hash_sourceid from one same source (see explanation further down)

In [ ]:
# SQL parameters
schema = "clean"
tables_dict = {
    'jocojcmhcdiagnoses': {
        "id": "patid", 
        "joco_id_col": "sourceid",
        "source": "'JOCOJCMHCDEMOGRAPHICS.PATID'"
        },
    'jocomedactincidents': {
        "id": "hash_rcdid",
        "joco_id_col": "hash_sourceid",
        "source": "'JOCOMEDACTINCIDENTS.RCDID'"
        },
    'joco110hsccclientmisc2eaimpression': {
        "id": "clientid",
        "joco_id_col": "sourceid",
        "source": "'JOCO110HSCCCLIENT2.CLIENTID'"
    },
    'jocojcmexoverdosessuicides': {
        "id": "id", 
        "joco_id_col": "sourceid",
        "source": "'JOCOJCMEXOVERDOSESSUICIDES.ID'"
    },
    'jocodcmexoverdosessuicides': {
        "id": "casenum", 
        "joco_id_col": "sourceid",
        "source": "'JOCODCMEXOVERDOSESSUICIDES.CASENUM'"
    },
    'joco110hsccclientmisc2eadiagnosis': {
        "id": "clientid", 
        "joco_id_col": "sourceid",
        "source": "'JOCO110HSCCCLIENT2.CLIENTID'"
    },
    'jocojimsmnhdata': {
        "id": "mni", 
        "joco_id_col": "sourceid",
        "source": "'JOCOJIMSNAMEINDEX.MNI_NO_0'"
    },
}


In [ ]:
# SQL Query
sql_q_empty = """
			with source_table_count as (
				select 
					count(distinct {}) as unique_id_count,
					count({}) as id_count
				from {}.{}
			),
			joclient_count as (
					select 
						count(distinct {}) as unique_sourceid_count,
						count({}) as sourceid_count
					from {}.jocojococlient
					where "source" = {}
			),
			matched_count as (
					select count(distinct c.{}) as unique_matched_ids_count
					from {}.{} t
					inner join {}.jocojococlient c
					on t.{} = c.{}
					where c."source" = {}
			)
			select distinct
				'{}' as table_name,
				'{}' as unique_id,
				
				(
					select unique_id_count
					from source_table_count
				) as unique_id_count,
				(
					select id_count
					from source_table_count
				) as id_count,
				(
					select unique_sourceid_count
					from joclient_count
				) as unique_sourceid_count,
				(
					select sourceid_count
					from joclient_count
				) as sourceid_count,
				(
					select unique_matched_ids_count
					from matched_count
				) as unique_matched_ids_count,
				((
					select unique_matched_ids_count
					from matched_count
				) * 1.0 / (
					select unique_id_count
					from source_table_count
				)) as pct_matched;
			"""

In [ ]:
# Return counts of matches with jocojococlient table
counts_df = pd.DataFrame(
    columns = ["table_name", "unique_id", "unique_sourceid_count",
                "sourceid_count", "unique_id_count", "id_count"])

for table, vals in tables_dict.items():
    id = vals["id"]
    source = vals["source"]
    joco_id_col = vals["joco_id_col"]
    sql_q = sql_q_empty.format(id, id, schema, table, 
                                joco_id_col, joco_id_col, schema, source,
                                joco_id_col, schema, table, schema, id, joco_id_col, source, 
                                table, id)
    counts_df = pd.concat([counts_df,
                            pd.read_sql(sql_q, db_conn)])

counts_df

#### Columns above: 
* unique_id: name of id column in source table
* unique_source_id_count: count of unique ids in source table
* sourceid_count: count of ids in source table
* unique_id_count: count of unique ids in jocojococlient table related to the source table 
* id_count: count of ids in jocojococlient table related to the source table
* pct_matched: % of unique ids in the source table that appear in the jocojococlient table

#### General comments: 
* The matches are working well across key tables considered
* Almost all ids in the source tables are found in the jocojococlient table
* The tables where we see repeated ids the most are jocojcmhcdiagnoses, joco110hsccclientmisc2eaimpression, and joco110hsccclientmisc2eadiagnosis
* For the following columns, we have (almost) a one-to-one relationship: jocodcmexoverdosessuicides, jocojcmexoverdosessuicides, jocomedactincidents

#### Repeated ids:
* Some joids will match to several sources, meaning these individuals have interacted with several parts of the system. 
* As mentioned above, one same joid may match to multiple sourceid/hash_sourceid (e.g. patid for MHC data) within one source (e.g. JOCOJCMHCDEMOGRAPHICS.PATID). For example,joid 1 may be linked to the patid id A and patid id B. This may happen for several reasons:
    1. There may be an issue in the jocojococlient record matching (we may be matching individuals A and B to one same joid 1 because they have the same name, but they may not correspond to the same person).
    2. The source may be failing to match individuals to their ids, e.g. one individual calls MHC and is given the patid A, and then they complete an in-person MHC assessment  and are assigned the patid B. In this case, one same person has two different patids, but we're then able to link it to the same joid 1 thanks to the record matching proces in MyRC. 
    3. The source ids may not refer to individuals. For example, MHC may assign a patid per case rather than per individual.


Tables matched on sourceid:
* 14.3% joids appear in more than 1 source
* 4.5% of joids matched by sourceid have duplicate sourceids, meaning one same joid has more than 1 sourceid (e.g. patids)
* The most significant cases are found in JIMS data, with one joid being matched to 18 mni ids
    * The ids that are most repeated in JIMS are generally matched by first and last name, not by ssn and dob 
        * Is this matching reliable? ssn and dob data may not be very prevalent in JIMS data? 
        * Would we get different mni ids for different parts of the law enforcement system, e.g. different jails? 

Tables matched on hash_sourceid (MedACT):
* MedACT data is where we see the most cases of a joid matched to multiple hash_sourceid
* The most significant case is a joid being matched to 164 hash_sourceids
* This duplicity per joid in MedACT data is quite significant compared to other tables (the matching is also different) - we discussed they might not necessarily assign 1 id per person

Further data analysis for each source will be done separately, but the matches to jocojococlient appear to be working. 